# The Annotated Transformer

**References**: 
- The Annotated Transformer, 2018 (https://nlp.seas.harvard.edu/2018/04/03/attention.html)

**Purpose**: 
- The Attention is All You Need paper is very clearly written, but it's quite difficult to implement correctly.
- We want to improve translation quality of DL models as well as performance on NLP tasks.
- The goal of reducing sequential computation leads us to CNNs which compute hidden representations in parallel for all input and output positions. 
- The number of operations required to relate signals from two arbitrary input or output positions grows in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. 
- Problem: This makes it more difficult to learn dependencies between distant positions.


**Approach**: 
- The Annotated version of the paper has a line-by-line implementation.
- In the Transformer, the number of operations required to relate signals from two arbitrary input or output positions is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect they counteract with Multi-Head Attention.

**Result**: 
- Full-implementation which composes of 400 lines of library code which can process 27K tokens / sec on 4 GPUs.
- Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence aligned RNNs or convolution (enabling parallelized computation instead of sequential - which is critical when highly efficient training).



**Definitions**: 
Self-attention/intra-attention = an attention mechanism relating different positions of a single sequence in order to compute a representaiton of the sequence

**Notes**:
- Self-attention had been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment andl earning task-indepeendent sentence representations. 
    - Note: it looks like self-attention was not a new idea, it had been proven already. The key insight was the scope of the problem because "we need to learn dependencies between distant positions without needing the number of operations to relate signals to grow in distance between positions" + what's SOTA in solving this problem right now (which is self-attention).
    - In fact, E2E memory networks based on recurrent attention mechanism instead of sequence aligned recurrence have been shown to perform well on simple-language Q&A and language modeling tasks.
        - Note: looks like the recurrent attention mechanism team didn't realize that they don't need the recurrent hidden state and can just process things in parallel instead of sequentially. This is where perspective and scope of the problem you care about really matters in helping you explore different options - the motivations are completely different.
    - Another big takeaway is that typically you "could" just say that scale will win and eventually learn all of these things, but parallelization and higher efficiency was actually necessary here to realize those gains earlier on.

**FAQs**:

**Takeaways**:
1. Note: looks like the recurrent attention mechanism team didn't realize that they don't need the recurrent hidden state and can just process things in parallel instead of sequentially. This is where perspective and scope of the problem you care about really matters in helping you explore different options - the motivations are completely different.
2. Another big takeaway is that typically you "could" just say that scale will win and eventually learn all of these things, but parallelization and higher efficiency was actually necessary here to realize larger scale and those gains.

**Action items**:


In [9]:
# !pip install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl numpy matplotlib spacy torchtext seaborn 
!pip install numpy matplotlib spacy torchtext seaborn 
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/68.0 kB ? eta -:--:--
     ---------------------------------------- 68.0/68.0 kB 1.9 MB/s eta 0:00:00
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/14.9 MB ? 


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.6 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 25.2 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 16.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------  2.5/2.5 MB 80.3 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 39.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

In [11]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many other models
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        """Take in and process masked src and target sequences."""
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [12]:
class Generator(nn.Module):
    """Define standard linear + softmax generation step."""
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab) # logits (unnormalized logits for each token)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1) # log-probs

# Encoder and Decoder Stacks

## Encoder
The encoder is composed of a stack of N = 6 identical layers.

In [13]:
def clones(module, N):
    """Produce N identical layers."""
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [16]:
class Encoder(nn.Module):
    """Core encoder is a stack of N layers"""
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        """Pass the input (and mask) through each layer in turn."""
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [15]:
class LayerNorm(nn.Module):
    """Construct a layernorm module."""
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2 # elementwise affine transformation

In [ ]:
# assume d_model=512

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last. Also note: this is the classic method of a Pre-LayerNorm now which is empirically better than a Post-LayerNorm where
    def forward(self, x, sublayer):
        return self.norm(x + self.dropout(sublayer(x)))
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-wise fully connected feed-forward network.

In [ ]:
class EncoderLayer(nn.Module):
    "encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)) # MHA
        return self.sublayer[1](x, self.feed_forward) # FF